In [0]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import calendar

In [0]:
# AFSNT 파일로 date 뽑기
AFSNT = pd.read_csv('data/AFSNT.csv', encoding='CP949')
AFSNT_date = list(set(AFSNT['DATE']))
AFSNT_DLY = pd.read_csv('data/AFSNT_DLY.csv', encoding='CP949')
AFSNT_DLY_date = list(set(AFSNT['DATE']))
date = AFSNT_date + AFSNT_DLY_date
date.sort()
date_df = pd.DataFrame({'date':date})

In [0]:
AFSNT_ALL = pd.concat([AFSNT, AFSNT_DLY], ignore_index=True)
AFSNT_ALL = AFSNT_ALL[['DATE','SDT_DY']].drop_duplicates().reset_index(drop=True)
AFSNT_ALL.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,DATE,SDT_DY
0,2017-01-01,일
1,2017-01-02,월
2,2017-01-03,화
3,2017-01-04,수
4,2017-01-05,목


In [0]:
def findmdidx(s):
    try:
        idx1 = s.index("월")
        idx2 = s.index("일")
    return idx1,idx2

In [0]:
pat = "{0:0>4}-{1:0>2}-{2:0>2}"
year = [2017, 2018, 2019]
holi = []
for y in range(len(year)):
    url = 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query='+str(year[y])+'년+공휴일'
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.select('#dss_free_uio_cont1 > div > div > table > tbody')
    
    st = list(result[0].strings)
    tmp = []
    for i in range(len(st)):
        try:
            int(st[i][0])
        except:
            continue
    if "월" in st[i]:
        tmp.append(st[i])
          
    for i in range(len(tmp)):
        if '~' in tmp[i]:
            t = tmp[i].split(' ~ ')
            m1, d1 = findmdidx(t[0])
        try:
            m2, d2 = findmdidx(t[1])
        except:
            m2 = m1
            d2 = -1
        
        mon1 = int(t[0][:m1])
        if len(t[1])>3:
            mon2 = int(t[1][:m2])
        else:
            mon2 = mon1
        
        day1 = int(t[0][m1+2:d1])
        try:
            day2 = int(t[1][m2+2:d2])
        except:
            day2 = int(t[1][:-1])
        
        if mon1 != mon2:
            l1, last = calendar.monthrange(year[y], mon1)
            holi.append(pat.format(year[y], mon1, day1))
            holi.append(pat.format(year[y], mon2, day2))
        if last != day1:
            holi.append(pat.format(year[y], mon1, last))
        else:
            holi.append(pat.format(year[y], mon2, day2-1))
    else:
        for k in range(day1, day2+1):
            holi.append(pat.format(year[y], mon1, k))
          
    else:
        m, d = findmdidx(tmp[i])
        holi.append(pat.format(year[y], tmp[i][:m], tmp[i][m+2:d]))

In [0]:
holi.sort()
holi

['2017-01-01',
 '2017-01-27',
 '2017-01-28',
 '2017-01-29',
 '2017-01-30',
 '2017-03-01',
 '2017-05-03',
 '2017-05-05',
 '2017-05-09',
 '2017-06-06',
 '2017-08-15',
 '2017-10-02',
 '2017-10-03',
 '2017-10-03',
 '2017-10-04',
 '2017-10-05',
 '2017-10-06',
 '2017-10-09',
 '2017-12-25',
 '2018-01-01',
 '2018-02-15',
 '2018-02-16',
 '2018-02-17',
 '2018-03-01',
 '2018-05-05',
 '2018-05-07',
 '2018-05-22',
 '2018-06-06',
 '2018-06-13',
 '2018-08-15',
 '2018-09-23',
 '2018-09-24',
 '2018-09-25',
 '2018-09-26',
 '2018-10-03',
 '2018-10-09',
 '2018-12-25',
 '2019-01-01',
 '2019-02-04',
 '2019-02-05',
 '2019-02-06',
 '2019-03-01',
 '2019-05-05',
 '2019-05-06',
 '2019-05-12',
 '2019-06-06',
 '2019-08-15',
 '2019-09-12',
 '2019-09-13',
 '2019-09-14',
 '2019-10-03',
 '2019-10-09',
 '2019-12-25']

In [0]:
holi_bfr = [0]*len(AFSNT_ALL)
holi_day = [0]*len(AFSNT_ALL)
holi_aft = [0]*len(AFSNT_ALL)
for i in range(len(AFSNT_ALL)):
    if AFSNT_ALL['DATE'][i] in holi:
        holi_day[i] = 1
        if i-1 > 0 and AFSNT_ALL['DATE'][i-1] not in holi:
            if AFSNT_ALL['SDT_DY'][i-1] not in ['토', '일']:
                holi_bfr[i-1] = 1
        if i+1 < len(AFSNT_ALL) and AFSNT_ALL['DATE'][i+1] not in holi and AFSNT_ALL['SDT_DY'][i+1] not in ['토', '일']:
            holi_aft[i+1] = 1

In [0]:
AFSNT_ALL['holi_bfr'] = holi_bfr
AFSNT_ALL['holi_day'] = holi_day
AFSNT_ALL['holi_aft'] = holi_aft

In [0]:
AFSNT_ALL

,DATE,SDT_DY,holi_bfr,holi_day,holi_aft
0,2017-01-01,일,0,1,0
1,2017-01-02,월,0,0,1
2,2017-01-03,화,0,0,0
3,2017-01-04,수,0,0,0
4,2017-01-05,목,0,0,0
5,2017-01-06,금,0,0,0
6,2017-01-07,토,0,0,0
7,2017-01-08,일,0,0,0
8,2017-01-09,월,0,0,0
9,2017-01-10,화,0,0,0


In [0]:
len(AFSNT_ALL)

926

#### 징검다리연휴처리

In [0]:
holi_bfr_bigcon = holi_bfr
holi_day_bigcon = holi_day
holi_aft_bigcon = holi_aft

In [0]:
for i in range(len(AFSNT_ALL)):
    if AFSNT_ALL['holi_bfr'][i] == 1 and AFSNT_ALL['holi_aft'][i] == 1:
        holi_bfr_bigcon[i]=0
        holi_day_bigcon[i]=1
        holi_aft_bigcon[i]=0
    if i-1 > 0:
        if AFSNT_ALL['SDT_DY'][i-1] == '일' and AFSNT_ALL['holi_bfr'][i] == 1:
            holi_day_bigcon[i]=1
            holi_bfr_bigcon[i]=0
        if AFSNT_ALL['SDT_DY'][i] == '토' and AFSNT_ALL['holi_day'][i-1] == 1 and AFSNT_ALL['holi_day'][i+2] == 1:
            holi_day_bigcon[i]=1
            holi_bfr_bigcon[i]=0
            holi_aft_bigcon[i]=0
            holi_day_bigcon[i+1]=1
            holi_bfr_bigcon[i+1]=0
            holi_aft_bigcon[i+1]=0
    if i+1 < len(AFSNT_ALL):
        if AFSNT_ALL['SDT_DY'][i+1] == '토' and AFSNT_ALL['holi_aft'][i] == 1:
            holi_day_bigcon[i]=1
            holi_aft_bigcon[i]=0

In [0]:
AFSNT_ALL_bigcon = AFSNT_ALL[['DATE', 'SDT_DY']]
AFSNT_ALL_bigcon['holi_bfr'] = holi_bfr_bigcon
AFSNT_ALL_bigcon['holi_day'] = holi_day_bigcon
AFSNT_ALL_bigcon['holi_aft'] = holi_aft_bigcon

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
AFSNT_ALL_bigcon.to_csv("data/holiday.csv", index=False, encoding='CP949')